<a href="https://colab.research.google.com/github/JuanCervigon/Globalizacion/blob/main/figaro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

* Importar las librerías necesarias
* Leer figaro largo

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [2]:
# Conectar con Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Leer figaro
ruta='/content/drive/MyDrive/Figaro/flatfile_eu-ic-io_ind-by-ind_23ed_2021.csv'
figaro=pd.read_csv(ruta)

# Crear nuevas columnas figaro

* fila: mapeo geo + cnae
* columna: mapeo geo + cnae
* geo_fila: mapeo geo
* geo_columna: mapeo geo
* Tipo_celda: func vectorizada. ci, df, va, op
* XobsValue: np.where(refArea=counterpartArea, 0, obsValue)


In [4]:
# Crear fila, columna, geo_fila, geo_columna, tipo_celda, XobsValue

# Leer el fichero excel de mapeo
mapeo='/content/drive/MyDrive/Figaro/m_paises_21cnae.xlsx'

fgeo=pd.read_excel(mapeo,'fgeo').set_index('cod')
fnace=pd.read_excel(mapeo,'fnace').set_index('cod')
cgeo=pd.read_excel(mapeo,'cgeo').set_index('cod')
cnace=pd.read_excel(mapeo,'cnace').set_index('cod')

# Convertir convertir los mapeos en diccionarios

fgeo=dict(fgeo.to_dict()['map'])
fnace=dict(fnace.to_dict()['map'])
cgeo=dict(cgeo.to_dict()['map'])
cnace=dict(cnace.to_dict()['map'])

# Crear las nuevas columnas con .map()

figaro['fila']=figaro.refArea.map(fgeo)+'_'+figaro.rowIi.map(fnace)
figaro['columna']=figaro.counterpartArea.map(cgeo)+'_'+figaro.colIi.map(cnace)
figaro['geo_fila']=figaro.refArea.map(fgeo)
figaro['geo_columna']=figaro.counterpartArea.map(cgeo)

# Clasificar las celdas en ci, va, df y op

# Crear una función que devuelve el tipo de celda en función de refArea, colIi
def clasifica(refArea,colIi):
    tipo = (
          "op" if (refArea=='W2') & (colIi[:2]=='P3'or colIi[:2]=='P5')
    else  "va"  if (refArea=='W2')
    else  "df"  if (colIi[:2]=='P3'or colIi[:2]=='P5')
    else "ci"
    )

    return tipo

# Vectorizar la función
func = np.vectorize(clasifica)

# Crear la columna tipo_celda a partir de la función vectorizada
figaro["tipo_celda"] = func(figaro.refArea,figaro.colIi)

# Crear la columna XobsValue, que vale cero cuando geo_fila=geo_columna y obsValue en caso contrario
figaro['XobsValue']=np.where(figaro['refArea']==figaro['counterpartArea'],0,figaro['obsValue'])

In [ ]:
# Comprobar que se han creado las columnas
figaro.columns

Index(['icioiRow', 'icioiCol', 'refArea', 'rowIi', 'counterpartArea', 'colIi',
       'obsValue', 'fila', 'columna', 'geo_fila', 'geo_columna', 'tipo_celda',
       'XobsValue'],
      dtype='object')

# Guardar figaro en formato excel

In [ ]:
# Crear la matriz colapsando los valores por las fila y columna

figaro_matriz=figaro.pivot_table(values='obsValue', index="fila", columns="columna", aggfunc='sum',sort=False)

# Definir la ruta del fichero excel
ruta_excel='f_paises_21.xlsx'

# Guardar el fichero excel
figaro_matriz.to_excel(f'/content/drive/MyDrive/Figaro/{ruta_excel}')


# Extraer matrices

* Bilateral Trade Flows, pais-pais, BTR
* Bilateral Trade Flows, GRTR
* Valor añadido, w
* Producción total, x
* Ratio valor añadido por rama de actividad, v
* Consumos intermedios, z
* demanda final, y


## Bilateral Trade Flows, pais-pais, BTR

In [12]:
# Crear un dataframe con las exportaciones de geo_fila a geo_columna
BTR = figaro.loc[(figaro['tipo_celda']) == 'ci'].\
      groupby(['geo_fila','geo_columna'])['XobsValue'].\
      sum().reset_index()

## Bilateral Trade Flows, GRTF

Gross Total Trade Flows de Consumos Intermedios, Demanda Final y Total

* GRTF_INT, Gross Intermediate Trade Flows
* GRTF_FNL, Gross Final Trade Flows
* GRTF, Gross Total Trade Flows

In [17]:
# Seleccionar celdas de consumos intermedios
grtr_int_largo=figaro.loc[(figaro['tipo_celda']=='ci'),\
                    ['fila','geo_columna','XobsValue']]

# Crear una matriz agregando los 'obsValue' por 'ci'

grtr_int=grtr_int_largo.pivot_table(
         values='XobsValue', index="fila",
         aggfunc='sum',sort=False)

# Seleccionar celdas de demanda final
grtr_fnl_largo=figaro.loc[(figaro['tipo_celda']=='df'),
                    ['fila','geo_columna','XobsValue']]

# Crear una matriz agregando los 'XobsValue' por 'df'

grtr_fnl=grtr_fnl_largo.pivot_table(
         values='XobsValue', index="fila",
         aggfunc='sum',sort=False)

# Seleccionar celdas de consumos intermedios y demanda final
grtr_largo=figaro.loc[(figaro['tipo_celda']=='ci')|(figaro['tipo_celda']=='df'),
                    ['fila','geo_columna','XobsValue']]

# Crear una matriz agregando los 'XobsValue' por 'ci'+'df'

grtr=grtr_largo.pivot_table(
          values='XobsValue', index="fila",
          aggfunc='sum',sort=False)


## Valor añadido, w

* w = impuestos producción+ remuneración asalariados+rentas mixtas
* w = D21X31 + D1 + B2A3G
* Dimensión  1 x 𝑁𝐾

Hay que hacer dos operaciones sobre `figaro`:

* Filtrar las celdas de valor añadido (.loc)
* Crear la matriz (pivot_table)


In [21]:
w_largo=figaro.loc[(figaro['tipo_celda']=='va'),\
                   ['tipo_celda','columna','obsValue']]

w=w_largo.pivot_table(values='obsValue', index="tipo_celda", \
                                columns='columna', \
                                aggfunc='sum',sort=False)


## Producción Total, x

La producción total por ramas de actividad es la suma de las columnas (o de las filas) de la matriz de figaro

* x = suma de las columnas. Consumos Intermedios + Valor añadido
* Dimensión 1 x $NK$
* Alternativamente: x = suma de las filas. Consumos Intermedios + demanda final


Hay que hacer dos operaciones sobre `figaro`:

* Filtrar las celdas de Consumos Intermedios y Valor añadido (.loc)
* Crear la matriz (pivot_table) sumando obsValue por filas y columnas
* Sumar todas las columnas (sum(axis=0)

In [22]:
x_largo=figaro.loc[
  (figaro['tipo_celda']=='ci') |(figaro['tipo_celda']=='va'),
  ['fila','columna','obsValue']]

ci_va=x_largo.pivot_table(values='obsValue', index="fila", \
                                columns='columna', \
                                aggfunc='sum',sort=False)
x=ci_va.sum(axis=0)

## Ratio valor añadido por rama de actividad, v

Es el cociente entre el valor añadido y el total producido, w/x


Dimension: $1$x$NK$

In [74]:
# Calcular v=w/x. Devuelve un dataframe
v = np.divide(w, x)
v.fillna(0, inplace=True)

## Consumos intermedios, z

* Seleccionar las celdas de consumos intermedios y agrupar por fila y columna
* Dimensión  𝑁𝐾 x 𝑁𝐾

In [24]:
z_largo=figaro.loc[
  (figaro['tipo_celda']=='ci'), ['fila','columna','obsValue']]

  # Crear la matriz
z=z_largo.pivot_table(values='obsValue', index="fila",
          columns='columna', aggfunc='sum',sort=False)

## Demanda final, y

* Es la extracción directa de las celdas de la demanda final
* Seleccionar las celdas 'df'
* Agrupar por fila y columna
* Dimensión 𝑁𝐾 x (𝑁𝑥5)



In [25]:
y_largo=figaro.loc[
  (figaro['tipo_celda']=='df'), ['fila','columna','obsValue']]

y=y_largo.pivot_table(values='obsValue', index="fila",
          columns='columna', aggfunc='sum',sort=False)


# Calculo de indicadores

## Inversa de Leontief

* B=(I-A)-1, donde A es la matriz de coeficientes

In [69]:
# Calcular la inversa de Leontieff

A = np.divide(z, x) # Calcular los coeficientes
A.fillna(0, inplace=True) # Sustituir los NaN por ceros
I_menos_A=np.identity(A.shape[0])-A # Calcular I-A
B = inv(I_menos_A) # Calcular B =  inversa de I-A

In [71]:
# Convertir la matriz B en un dataframe con las filas y columnas de A
B = pd.DataFrame(B, columns=A.columns, index=A.index)

## Valor añadido generado por las exportaciones

Es el producto de tres matrices, ratio va/producción x Leontief x GRTR diagonalizada

* Fórmula $v*B*GRTR$_$FNL$
* dimensión 1xNK = (1xNK)(NKxNK)(NKxNK)




In [75]:
# Filtrar las filas y columnas de v, B y grtr empiezan por "ES"
B_ES = B.filter(like='ES', axis=0).filter(like='ES', axis=1)
v_ES = v.filter(like='ES', axis=1)
grtr_ES = grtr.filter(like='ES', axis=0)

In [76]:
# Crear una matriz de ceros con grtr en la diagonal
grtr_ES_d = np.zeros((len(grtr_ES),len(grtr_ES)))
diagonal=grtr_ES.values
np.fill_diagonal(grtr_ES_d, diagonal)

In [87]:
# Calcular el contenido de valor añadido de las exportaciones españolas por rama
# Multiplicar las matrices v_ES , B_ES y grtr_ES_d

va_ES_exp = np.dot(np.dot(v_ES, B_ES), grtr_ES_d)


In [89]:
va_ES_exp = pd.DataFrame(va_ES_exp, columns=v_ES.columns).transpose()
va_ES_exp


,0
columna,
ES_A,17094.959111
ES_B,2190.639697
ES_C,124877.444865
ES_D,782.673683
ES_E,1785.265789
ES_F,2013.013799
ES_G,51739.789439
ES_H,16575.797129
ES_I,5556.455438


# Grafico coropletico con BTR

In [ ]:
pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.8 MB/s eta 0:00:00


In [ ]:
import pycountry

# crear un diccionario "clave:valor = alpha_2:alpha_3"

alphas={pais.alpha_2:pais.alpha_3 for pais in pycountry.countries}
alphas

In [ ]:
# Añadir la columna alpha_3 a BTR

BTR['alpha_3']=BTR['geo_columna'].map(alphas)

In [ ]:
import plotly.express as px


# Seleccionar el pais
pais='CN'

BTR_ES=BTR.loc[BTR['geo_fila']==pais]

# Crear el gráfico coroplético
fig = px.choropleth(BTR_ES,
      locations="alpha_3",
      color="XobsValue",
      hover_name="alpha_3",
      color_continuous_scale=px.colors.sequential.Plasma)

fig.update_layout(title_text=f'Exportaciones de {pais}', title_font=dict(size=16))
fig.show()